In [ ]:
%pip install pandas sqlalchemy psycopg2-binary matplotlib python-dotenv

In [ ]:
import os
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv

load_dotenv()

DB_USER = os.getenv("DB_USER")
DB_PASS = os.getenv("DB_PASS")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")

engine = create_engine(f'postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}')

try:
    with engine.connect() as conn:
        print("Successfully connected to the database securely!")
except Exception as e:
    print(f"Connection failed: {e}")

In [ ]:
query = """
SELECT 
    d.date_id AS "date",
    ha.happy_mood AS "good day",
    ha.productive,
    he.weight_loss AS "lost weight",
    he.worked_out AS "exercised",
    he.took_vitamins AS "took supplements",
    he.ate_healthy AS "ate healthy",
    ga.health_improvements_made AS "healthy habits",
    ga.read,
    ga.studied_ou,
    ga.studied_de,
    ga.studied_french
FROM metrics_table AS mt
INNER JOIN date AS d
    ON mt.date_id = d.date_id
INNER JOIN happiness AS ha
    ON mt.happiness_metric_id = ha.happiness_metric_id
INNER JOIN health AS he
    ON mt.health_metric_id = he.health_metric_id
INNER JOIN goals_achieved AS ga
    ON mt.goal_metric_id = ga.goal_metric_id
"""

df = pd.read_sql(query, engine)